In [1]:
#STEP 1: Record a 5-Second Selfie Video
import cv2
import time

# Initialize video capture
cap = cv2.VideoCapture(0)
fps = 20
duration = 5  # seconds
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

out = cv2.VideoWriter('selfie_video.avi', cv2.VideoWriter_fourcc(*'XVID'), fps, (frame_width, frame_height))

start_time = time.time()
print("Recording started...")

while int(time.time() - start_time) < duration:
    ret, frame = cap.read()
    if not ret:
        break
    out.write(frame)
    cv2.imshow("Recording... Press 'q' to stop", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

print("Recording finished.")

cap.release()
out.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 

ImportError: numpy._core.multiarray failed to import

In [3]:
#STEP 2: Extract Frames from the Video
import os

video_path = 'selfie_video.avi'
output_folder = 'frames'
os.makedirs(output_folder, exist_ok=True)

cap = cv2.VideoCapture(video_path)
count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame_path = os.path.join(output_folder, f'frame_{count:03d}.jpg')
    cv2.imwrite(frame_path, frame)
    count += 1

cap.release()
print(f"{count} frames extracted.")


130 frames extracted.


In [4]:
#STEP 3: Skin Detection on Frames

def detect_skin(frame):
    # Convert to HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Define skin color range
    lower = (0, 48, 80)
    upper = (20, 255, 255)

    # Mask skin
    skin_mask = cv2.inRange(hsv, lower, upper)
    skin = cv2.bitwise_and(frame, frame, mask=skin_mask)

    return skin


In [5]:
#STEP 4: Apply Skin Detection to All Frames
    
processed_folder = 'processed_skin_frames'
os.makedirs(processed_folder, exist_ok=True)

for fname in os.listdir(output_folder):
    path = os.path.join(output_folder, fname)
    frame = cv2.imread(path)
    skin = detect_skin(frame)
    save_path = os.path.join(processed_folder, fname)
    cv2.imwrite(save_path, skin)

print("Skin regions extracted and saved.")


Skin regions extracted and saved.


In [16]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, Dropout, BatchNormalization, Concatenate, Multiply, Add, Lambda
from tensorflow.keras.applications import ResNet101, DenseNet121
import tensorflow as tf

input_shape = (224, 224, 3)
input_tensor = Input(shape=input_shape)

resnet_base = ResNet101(include_top=False, weights='imagenet', input_shape=input_shape)
x = resnet_base(input_tensor)
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu')(x)

densenet_base = DenseNet121(include_top=False, weights='imagenet', input_shape=input_shape)
y = densenet_base(input_tensor)
y = GlobalAveragePooling2D()(y)
y = Dense(512, activation='relu')(y)
y = BatchNormalization()(y)
y = Dense(256, activation='relu')(y)
y = BatchNormalization()(y)
y = Dropout(0.3)(y)
y = Dense(128, activation='relu')(y)

concat = Concatenate()([x, y])
attn_weights = Dense(2, activation='softmax')(concat)

alpha_x = Lambda(lambda z: tf.expand_dims(z[:, 0], axis=-1), output_shape=(1,), name="alpha_x")(attn_weights)
alpha_y = Lambda(lambda z: tf.expand_dims(z[:, 1], axis=-1), output_shape=(1,), name="alpha_y")(attn_weights)

x_scaled = Multiply()([x, alpha_x])
y_scaled = Multiply()([y, alpha_y])

combined = Add()([x_scaled, y_scaled])

combined = Dense(256, activation='relu')(combined)
combined = BatchNormalization()(combined)
combined = Dropout(0.3)(combined)

combined = Dense(128, activation='relu')(combined)
combined = BatchNormalization()(combined)
combined = Dropout(0.3)(combined)

output = Dense(6, activation='softmax')(combined)

model = Model(inputs=input_tensor, outputs=output)


In [17]:
model.load_weights("skin_disease.h5") 

In [18]:
#STEP 6: Define Prediction Function
from tensorflow.keras.preprocessing.image import img_to_array
import numpy as np

IMG_SIZE = 224
class_names = ['normal', 'Eczema', 'Acne', 'Rosacea', 'Actinic Keratosis', 'Basal Cell Carcinoma']  # Update as per your model

def predict_frame(image):
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    image = img_to_array(image) / 255.0
    image = np.expand_dims(image, axis=0)
    prediction = model.predict(image)
    class_id = np.argmax(prediction)
    return class_names[class_id], prediction[0][class_id]


In [19]:
#STEP 7: Run Predictions on Each Processed Frame
for fname in sorted(os.listdir(processed_folder)):
    path = os.path.join(processed_folder, fname)
    img = cv2.imread(path)
    label, confidence = predict_frame(img)
    print(f"{fname}: {label} ({confidence:.2f})")


1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step
frame_000.jpg: normal (0.81)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 248ms/step
frame_001.jpg: normal (0.81)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 205ms/step
frame_002.jpg: normal (0.81)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 225ms/step
frame_003.jpg: normal (0.81)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 219ms/step
frame_004.jpg: normal (0.81)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 236ms/step
frame_005.jpg: normal (0.81)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 245ms/step
frame_006.jpg: normal (0.81)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 267ms/step
frame_007.jpg: normal (0.81)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 268ms/step
frame_008.jpg: normal (0.81)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 244ms/step
frame_009.jpg: normal (0.81)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 279ms/step
frame_010.jpg: normal (0.81)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 246ms/step
frame_011.jpg: normal (0.81)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 234ms/step
frame_012.jpg: normal (0.81)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 268ms/step
frame_013.jpg: normal (0.81)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 255ms/step
frame_014.jp

In [20]:
#Final Prediction via Majority Voting
from collections import Counter

predictions = []

for fname in sorted(os.listdir(processed_folder)):
    path = os.path.join(processed_folder, fname)
    img = cv2.imread(path)
    label, _ = predict_frame(img)
    predictions.append(label)

final_prediction = Counter(predictions).most_common(1)[0]
print(f"\nFinal Predicted Skin Condition: {final_prediction[0]} ({final_prediction[1]} frames)")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 225ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 216ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 232ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 218ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 224ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 216ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 226ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 196ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 239ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 198ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 204ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 210ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 207ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 216ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 215ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 240ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 209ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 251ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 202ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 236ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 243ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 230ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 